# PLAYGROUND NOTEBOOK
Notebook untuk mengecek kode yang telah dibuat

In [2]:
import pandas as pd
from utils import load_model_and_tokenizer

llama32_results = pd.read_json("meta-llama_Llama-3.2-1B-Instruct-50_results.json")
model_name = "meta-llama/Llama-3.2-1B-Instruct"
model, tokenizer, config = load_model_and_tokenizer(model_name)

Loading model on cuda with torch.float16


## Cek Prioritas Memilih Token *Ground-Truth*

mengambil contoh row dimana jawaban *ground-truth*-nya panjang

In [ ]:
from retrieve_logprobs import retrieve_correct_token_logprobs

correct_token_logprobs = retrieve_correct_token_logprobs(llama32_results, tokenizer)